In [ ]:
import warnings
import FunctionLib as FL
import inspect
from tqdm import tqdm
import astropy
import wave
from matplotlib.image import resample
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import re
import scipy

mpl.rcParams['font.family'] = 'serif'


warnings.filterwarnings("ignore")

DJAv4Catalog=FL.Spectrum_Catalog()
DJAv4Catalog.load_from_pkl(os.path.expanduser('~/DustCurve/spectrum_catalog.pkl'))

In [1]:
import warnings

import astropy.units
import FunctionLib as FL
import inspect
from tqdm import tqdm
import astropy
import wave
from matplotlib.image import resample
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import re
import scipy

mpl.rcParams['font.family'] = 'serif'


warnings.filterwarnings("ignore")

DJAv4Catalog=FL.Spectrum_Catalog()
DJAv4Catalog.load_from_pkl(os.path.expanduser('~/DustCurve/spectrum_catalog.pkl'))

beta=list()
halpha=list()
hbeta=list()

i=0
for id_subid, catalog in tqdm(DJAv4Catalog.catalog_iterator()):
    if not catalog.get('prism_filepath'):
        continue
    if catalog.get('file_count')<3:
        continue
    if catalog.get('prism_redshift') is None:
        continue


    if catalog['prism_redshift'] <3.8:
        continue
    if catalog['prism_redshift'] > 6.5:
        continue

    spectrum=FL.Load_Spectrum_From_Fits(catalog['prism_filepath'], redshift=catalog['prism_redshift'])


    #fit continuum
    spectrum.set_boundarys(1268*astropy.units.Angstrom, 2580*astropy.units.Angstrom)
    Fitter=FL.SpectralLineFitter(spectrum, 4863*astropy.units.Angstrom)
    exp_fit_result=Fitter.fit_power_law()
    if exp_fit_result.get('success') is not True:
        print('Failed to fit power law for', id_subid)
        continue

    #fit emission lines
    for available_filter in catalog['available_filters']:
        if available_filter=='prism-clear':
            continue

        try:
            #Halpha
            spectrum= FL.Load_Spectrum_From_Fits(catalog['grating_filepaths'][available_filter], redshift=catalog['prism_redshift'])
            Halpha_restframe_wavelength=6565*astropy.units.Angstrom
            spectrum.set_boundarys(6500*astropy.units.Angstrom, 6600*astropy.units.Angstrom)
            Fitter=FL.SpectralLineFitter(spectrum, Halpha_restframe_wavelength)

            initial_guess_index=np.argmin(np.abs(spectrum.processing_wavelengths.data-6565))

            initial_guess=[spectrum.processing_flux.data[initial_guess_index],spectrum.processing_wavelengths.data[initial_guess_index], 2,np.median(spectrum.processing_flux.data)]

            halpha_fit_result=Fitter.fit_single_gaussian_with_offset(initial_guess)
            if halpha_fit_result.get('success') is not True:
                continue
            if halpha_fit_result['parameters']['stddev'] > 2* astropy.units.Angstrom:
                continue
            if halpha_fit_result['parameters']['amplitude'] <= 1e-18* astropy.units.erg / astropy.units.cm**2 / astropy.units.s / astropy.units.Angstrom:
                continue
            if Fitter.check_line(Halpha_restframe_wavelength,halpha_fit_result['parameters']['mean']):
                halpha_flux=halpha_fit_result['integrated_flux']



        except Exception as e:
            # print('Failed to fit Halpha for', id_subid, e)
            continue

        try:
            #Hbeta
            spectrum= FL.Load_Spectrum_From_Fits(catalog['grating_filepaths'][available_filter], redshift=catalog['prism_redshift'])
            Hbeta_restframe_wavelength=4863*astropy.units.Angstrom
            spectrum.set_boundarys(4800*astropy.units.Angstrom, 4900*astropy.units.Angstrom)
            Fitter=FL.SpectralLineFitter(spectrum, Hbeta_restframe_wavelength)

            initial_guess_index=np.argmin(np.abs(spectrum.processing_wavelengths.data-4863))

            initial_guess=[spectrum.processing_flux.data[initial_guess_index],spectrum.processing_wavelengths.data[initial_guess_index], 2,np.median(spectrum.processing_flux.data)]

            hbeta_fit_result=Fitter.fit_single_gaussian_with_offset(initial_guess)
            if hbeta_fit_result.get('success') is not True:
                continue
            if hbeta_fit_result['parameters']['stddev'] > 2* astropy.units.Angstrom:
                continue
            if hbeta_fit_result['parameters']['amplitude'] <= 1e-18* astropy.units.erg / astropy.units.cm**2 / astropy.units.s / astropy.units.Angstrom:
                continue
            if Fitter.check_line(Hbeta_restframe_wavelength,hbeta_fit_result['parameters']['mean']):
                hbeta_flux=hbeta_fit_result['integrated_flux']

        except Exception as e:
            # print('Failed to fit Hbeta for', id_subid, e)
            continue



    if 'halpha_flux' in locals() and 'hbeta_flux' in locals():
        beta.append(exp_fit_result['parameters']['decay'])
        halpha.append(halpha_flux.to('erg / (cm2 s)'))
        hbeta.append(hbeta_flux.to('erg / (cm2 s)'))



    i+=1
    if i>1000:
        break

FileNotFoundError: The file /Users/xingyaocai/DustCurve/spectrum_catalog.pkl does not exist.